#### **DESARROLLO Y PRUEBA DE LOS ENDPOINTS**
Se desarrolla cada endpoint de manera aislada, probando sus respuestas y asegurando su efectividad antes de integrarlos en el archivo `main.py`.

Se utiliza el dataset procesado previamente.

In [10]:
import pandas as pd

In [11]:
# Cargamos el dataset que se procesó previamente (parquet)
movies_df = pd.read_parquet('movies_dataset_final.parquet')

- Cantidad de filmaciones por mes:

In [15]:
def cantidad_filmaciones_mes(mes: str) -> dict:
    """
    Devuelve la cantidad de películas que fueron estrenadas en el mes consultado.
    
    Parámetros:
    mes (str): Nombre del mes en español.
    
    Retorno:
    dict: Mensaje con la cantidad de películas estrenadas en el mes.
    """
    # Diccionario de meses en español a números
    meses_diccionario = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    # Convertimos el mes a su número correspondiente
    mes_numero = meses_diccionario.get(mes.lower())
    
    if mes_numero:
        # Filtramos las películas del mes del 'release_date'
        cantidad = movies_df[movies_df['release_date'].dt.month == mes_numero].shape[0]
        return {"mensaje": f"{cantidad} películas fueron estrenadas en el mes de {mes.capitalize()}"}
    else:
        return {"error": "Mes no válido. Por favor, ingrese en español un mes válido."}


In [17]:
cantidad_filmaciones_mes("enero")

{'mensaje': '36901 películas fueron estrenadas en el mes de Enero'}

- Cantidad de filmaciones por día:

In [20]:
def cantidad_filmaciones_dia(dia: str) -> dict:
    """
    Devuelve la cantidad de películas que fueron estrenadas en el día de la semana consultado.
    
    Parámetros:
    - dia (str): Nombre del día en español.
    
    Retorno:
    - dict: Mensaje con la cantidad de películas estrenadas en el día.
    """
    # Diccionario de días de la semana en español a su número correspondiente (lunes=0, domingo=6)
    dias_diccionario = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    
    # Convertimos el día a su número correspondiente
    dia_numero = dias_diccionario.get(dia.lower())
    
    if dia_numero is not None:
        # Filtramos las películas por el día de la semana del 'release_date'
        cantidad = movies_df[movies_df['release_date'].dt.dayofweek == dia_numero].shape[0]
        return {"mensaje": f"{cantidad} películas fueron estrenadas el día {dia.capitalize()}"}
    else:
        return {"error": "Por favor, ingrese en español un día válido."}

In [21]:
cantidad_filmaciones_dia('jueves')

{'mensaje': '71356 películas fueron estrenadas el día Jueves'}

- Por el título de la película y score:

In [22]:
def score_titulo(titulo: str) -> dict:
    """
    Devuelve el título, año de estreno y score de una filmación.

    Parámetros:
    - titulo (str): Título de la filmación.

    Retorno:
    - dict: Mensaje con el título, año de estreno y el score de la película.
    """
    # Filtramos la película por el título
    film = movies_df[movies_df['title'].str.lower() == titulo.lower()]
    
    if not film.empty:
        # Extraemos el año de estreno y el score
        titulo_film = film['title'].values[0]
        año_estreno = film['release_year'].values[0]
        score = film['vote_average'].values[0]
        
        return {
            "mensaje": f"La película '{titulo_film}' fue estrenada en el año {año_estreno} con un score de {score}"
        }
    else:
        return {"error": f"No se encontró la película con el título '{titulo}'."}


In [37]:
score_titulo('iron man')

{'mensaje': "La película 'Iron Man' fue estrenada en el año 2008 con un score de 7.4"}

- Por el título de la película y valoraciones:

In [38]:
def votos_titulo(titulo: str) -> dict:
    """
    Devuelve la cantidad de votos y el promedio de votaciones de una película.
    
    Parámetros:
    - titulo (str): Título de la película.

    Retorno:
    - dict: Mensaje con la cantidad de votos y el promedio, o un mensaje si no cumple el mínimo de votos.
    """
    # Filtramos la película por el título
    film = movies_df[movies_df['title'].str.lower() == titulo.lower()]
    
    if not film.empty:
        votos = film['vote_count'].values[0]
        promedio_votos = film['vote_average'].values[0]
        
        # Verificamos si la película tiene al menos 2000 votos
        if votos >= 2000:
            return {
                "mensaje": f"La película '{titulo}' tiene {votos} valoraciones, con un promedio de {promedio_votos}"
            }
        else:
            return {
                "mensaje": f"La película '{titulo}' no cumple con el mínimo de 2000 votos. Solo tiene {votos} votos."
            }
    else:
        return {"error": f"No se encontró la película con el título '{titulo}'."}


In [39]:
votos_titulo('iron man')

{'mensaje': "La película 'iron man' tiene 8951.0 valoraciones, con un promedio de 7.4"}

- Por el nombre del actor:

In [41]:
def get_actor(nombre: str) -> dict:
    """
    Devuelve información sobre un actor, incluyendo la cantidad de películas en las que ha participado
    y el retorno promedio de sus películas.

    Parámetros:
    - nombre_actor (str): Nombre del actor.

    Retorno:
    - dict: Mensaje con el éxito del actor, la cantidad de películas y el retorno promedio.
    """
    # Filtramos las películas en las que ha actuado el actor
    peliculas_actor = movies_df[movies_df['cast'].str.contains(nombre, case=False, na=False)]
    
    if not peliculas_actor.empty:
        cantidad_peliculas = peliculas_actor.shape[0]
        retorno_total = peliculas_actor['return'].sum()
        promedio_retorno = peliculas_actor['return'].mean()
        
        return {
            "mensaje": f"El actor '{nombre}' ha participado en {cantidad_peliculas} películas, "
                       f"con un retorno total de {retorno_total} y un retorno promedio de {promedio_retorno:.2f}."
        }
    else:
        return {"error": f"No se encontraron películas en las que haya actuado '{nombre}'."}


In [42]:
get_actor('Ben Affleck')

KeyError: 'cast'